# Custom MCP Server & Client Implementation

This notebook demonstrates building **custom MCP server and client implementations** using a trading account system. The key learning: MCP abstracts tool definitions from execution logic, enabling portable, reusable tool ecosystems.

**Architecture:**
1. **Business Logic Layer** (`accounts.py`): Core functionality
2. **MCP Server** (`accounts_server.py`): Exposes functions as MCP tools
3. **MCP Client** (`accounts_client.py`): Consumes tools programmatically
4. **Agent Integration**: Agents use MCP tools via OpenAI Agents SDK

In [ ]:
# Import dependencies
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown
from accounts import Account
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

load_dotenv(override=True)

## Phase 1: Test Business Logic Directly

In [ ]:
# Direct Python API Usage
account = Account.get("TestUser")
account

In [ ]:
# Execute Trade
account.buy_shares("AMZN", 3, "Testing the trading system")
account.report()

In [ ]:
# View Transaction History
account.list_transactions()

## Phase 2: Use Custom MCP Server

In [ ]:
# Connect to MCP Server
params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

## Phase 3: Agent with MCP Tools

In [ ]:
# Configure Agent
instructions = "You manage client accounts. Answer questions about balances and holdings."
request = "My name is TestUser and my account is under TestUser. What's my balance and holdings?"
model = "gpt-4o-mini"

# Execute via MCP
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(
        name="account_manager", 
        instructions=instructions, 
        model=model, 
        mcp_servers=[mcp_server]
    )
    with trace("account_query"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

## Phase 4: Custom MCP Client (Programmatic)

Building a client that interacts with MCP servers programmatically, without spawning agents.

In [ ]:
# List Tools via Client
mcp_tools = await list_accounts_tools()
print(f"MCP Tools: {mcp_tools}")

# Get OpenAI-compatible Tool Schemas
openai_tools = await get_accounts_tools_openai()
print(f"OpenAI Tools: {openai_tools}")

In [ ]:
# Agent with Custom Client Tools
request = "My name is TestUser. What's my balance?"

with trace("client_integration"):
    agent = Agent(
        name="account_manager", 
        instructions=instructions, 
        model=model, 
        tools=openai_tools
    )
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

## Phase 5: MCP Resources (Context Injection)

In [ ]:
# Read Resource (Account State)
context = await read_accounts_resource("TestUser")
print(context)

In [ ]:
# Verify via Direct API
Account.get("TestUser").report()